# 非卷积的图像分类

先前我们对图像分类都包含了 Conv2D 这个卷积层操作，实际上有很多方式都可以完成图像分类的任务。常见的方法有以下几种：  
* 包含 Conv2D 的卷积网络。无论是 Inception 结构还是 ResNet 都含有卷积层。  
* 只使用 Dense 全连接层的网络。
* 胶囊网络。  
* 多层 Encoder编码 和 Decoder 解码网络。Transformer就属于这一类。  
很多博客在描述图像分类任务的时候会先介绍使用全连接的网络，因为在卷积网络出现之前人们能想到的就是将图像每个像素进行全连接，卷积网络出现之后深度学习就步入了爆发性增长。  

我之所以将全连接网络放在这么后面是因为卷积网络是图像任务处理的绝对主流，全连接网络固然结构简单，基本都是由 Dense 和 Drouput 层组成，但实际上产中基本不会单纯地使用全连接网络，通常只在教学阶段会作为先导引入卷积网络。  

并且如果你足够细心就会发现全连接网络的教学案例都只用了 mnist 这个数据集，手写数字识别数据集每个样本大小为 28\*28，所以用全连接网络还是能够承受的。 

设想这么一个场景：  
28\*28 的数据对每个像素进行单元为 64 的一层全连接，参数量为 28\*28\*64(权重w)+64(偏置值b)=50,240  
28\*28 的数据使用 64 个卷积核的卷积网络，参数为 640  
两者相差 78.5 倍。  
事实上我也不怎么喜欢计算参数总量，那么怎么才能快速获得参数信息呢？使用 .summary() 方法。  

In [1]:
from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing import sequence

import numpy as np
import matplotlib.pyplot as plt

## .summary() 方法使用

summary()方法能够得到模型参数的具体信息，细致到每一层，他的输出的后几行是我们关心的部分，包含了如下几个：  
* Total params：总参数
* Trainable params：可训练参数
* Non-trainable params：不可训练参数  
你可能会问为什么会出现不可训练参数呢？我们构造一个模型不就是为了让他能训练么？  

实际上，让你从零开始训练一个神经网络的需求是比较少的，除非特定任务，或者公司处于初创阶段。  
通常情况下我们会大量使用别人已经训练好的权重（这些部分将在 Use Extent Model 文件夹下展开），已经停赛的 ImageNet 竞赛给我们提供了大量优秀模型的同时还有他们的权重，当我们打算使用他们训练好的模型权重时就需要锁死某些层的参数不让他们参与训练，这样做的好处有以下几点：  
* 极大降低模型训练时间。一个参数从 0 训练到 10 和从 9.9 训练到 10 耗费的时间与次数显然不同。
* 减少自身数据集总量的要求。想要训练一个优秀的模型需要喂入大量数据，使用已经训练好的权重就可以提供少量数据，因为已有权重是由先前大量训练得到的。  

在 Use Extent Model 文件夹下有锁死权重的案例，但我仍然希望初学者按照顺序学习。

In [3]:
input_layer = Input(shape=(784, ))    # 784=28*28
dense_layer = Dense(64)(input_layer)

model = Model(inputs=input_layer, outputs=dense_layer)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                50240     
Total params: 50,240
Trainable params: 50,240
Non-trainable params: 0
_________________________________________________________________


In [4]:
input_layer = Input(shape=(28, 28, 1))
conv_layer = Conv2D(64, 3)(input_layer)

model = Model(inputs=input_layer, outputs=conv_layer)
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 64)        640       
Total params: 640
Trainable params: 640
Non-trainable params: 0
_________________________________________________________________


## 步入正题

在下面的例子中，我将展示如何只用 Dense 层对 MNIST 数据集进行分类

In [16]:
(x_train, y_train), (_, _) = mnist.load_data()

x_train = x_train / 255.0
x_train = np.reshape(x_train, (60000, 784))
y_train = to_categorical(y_train, num_classes=10)

#### 卷积层

In [20]:
def dense_block(x, filters):
    flowData = x
    for fl in filters:
        flowData = Dense(fl, activation='relu')(flowData)
    return flowData

In [25]:
input_layer = Input(shape=(784,))
dense_layers = dense_block(input_layer, [256, 128, 64, 32])
dense_out = Dense(10, activation='softmax')(dense_layers)

最后一层通常不用 def 进行构建，因为往往需要使用不同的激活函数

In [26]:
model = Model(inputs=input_layer, outputs=dense_out)

In [28]:
model.compile(loss='categorical_crossentropy', metrics='accuracy', optimizer='adam')

In [29]:
model.fit(x_train, y_train, verbose=1, batch_size=256, epochs=5)

Epoch 1/5
235/235 [==============================] - 2s 5ms/step - loss: 0.7873 - accuracy: 0.7631
Epoch 2/5
235/235 [==============================] - 1s 5ms/step - loss: 0.1465 - accuracy: 0.9557
Epoch 3/5
235/235 [==============================] - 1s 4ms/step - loss: 0.0930 - accuracy: 0.9713
Epoch 4/5
235/235 [==============================] - 1s 4ms/step - loss: 0.0666 - accuracy: 0.9800
Epoch 5/5
235/235 [==============================] - 1s 4ms/step - loss: 0.0467 - accuracy: 0.9855


之所以使用 Dense 模型也能在这个案例中获得很好很快的效果，那是因为样本太小了，模型也不够复杂。  

实际始生产中一定要避免有这种 “用全连接解决图像问题” 的想法。  

但全连接网络在基于特征的数据分类上是一个不错的选择。